In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from sagemaker.pytorch import PyTorchModel
from datetime import datetime

In [3]:
region = boto3.Session().region_name
session = sagemaker.Session()

BUCKET = 'covid-19-image-dataset'
TRAIN_DATA_PREFIX = 'radiography_train_data'
TEST_DATA_PREFIX = 'radiography_test_data'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)
s3 = boto3.resource('s3')

In [4]:
train_data_s3 = 's3://{}/{}'.format(BUCKET, TRAIN_DATA_PREFIX)

## SageMaker Training

In [5]:
src_code_s3 = 's3://{}/{}'.format(BUCKET, 'training_jobs')
training_job_output_s3 = 's3://{}/{}'.format(BUCKET, 'training_jobs_output')

In [5]:
# Encapsulate training on SageMaker with PyTorch:
train_estimator = PyTorch(entry_point='transfer_learning_resnet.py',
                          role=role,
                          framework_version='1.8.1',
                          py_version='py3',
                          debugger_hook_config=False,
                          instance_count=1,
                          instance_type='ml.p3.2xlarge',
                          output_path=training_job_output_s3,
                          code_location=src_code_s3,
                          hyperparameters={'epochs': 2,
                                           'lr': 0.001,
                                           'batch-size': 8
                          }
)

In [6]:
# Setting up File-system to import data from S3
data_channels = {'train': sagemaker.inputs.TrainingInput(
                          s3_data_type='S3Prefix',
                          s3_data=train_data_s3,
                          content_type='image/jpeg',
                          input_mode='File'),
                 
                 'val': sagemaker.inputs.TrainingInput(
                        s3_data_type='S3Prefix',
                        s3_data=train_data_s3,
                        content_type='image/jpeg',
                        input_mode='File')
}

In [7]:
# Launching SageMaker training job
job_name = 'sagemaker-jam-covid19-' + str(datetime.now().strftime("%H-%M-%S-%f"))
print ('Launching Training Job:', job_name)
train_estimator.fit(inputs=data_channels, job_name=job_name)

Launching Training Job: sagemaker-jam-covid19-10-06-44-394219
2021-10-08 10:06:44 Starting - Starting the training job...
2021-10-08 10:07:08 Starting - Launching requested ML instancesProfilerReport-1633687604: InProgress
......
2021-10-08 10:08:08 Starting - Preparing the instances for training......
2021-10-08 10:09:15 Downloading - Downloading input data..................
2021-10-08 10:12:09 Training - Downloading the training image...
2021-10-08 10:12:37 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-08 10:12:37,870 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-08 10:12:37,894 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-08 10:12:39,326 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-08 10:12:39,716 sag

# WIP: SageMaker Inference

In [6]:
inference_job_output_s3 = 's3://{}/{}'.format(BUCKET, 'inference_jobs_output')
inference_input_data_s3 = 's3://{}/{}'.format(BUCKET, 'radiography_test_data')

In [8]:
#JOB_NAME = job_name
JOB_NAME = 'sagemaker-jam-covid19-09-13-03-552177'
inference_model_artefact_s3 = 's3://{}/{}/{}/{}'.format(BUCKET, 'training_jobs_output', JOB_NAME, 'output/model.tar.gz')

In [9]:
prediction_model = PyTorchModel(entry_point = 'transfer_learning_resnet.py',
                           model_data = inference_model_artefact_s3,
                           role = role,
                           framework_version = '1.8.0',
                           py_version = 'py3') 

## Real-time Endpoint

In [ ]:
predictor = prediction_model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

--

In [ ]:
predictions = predictor.predict('sample_image.png')

## Batch Transform

In [ ]:
prediction_transformer = prediction_model.transformer(instance_count = 1,
                                                      instance_type = 'ml.g4dn.xlarge',
                                                      strategy = 'MultiRecord',
                                                      max_concurrent_transforms = 4,
                                                      output_path = inference_job_output_s3)
prediction_transformer.transform(data = inference_input_data_s3,
                                 data_type = 'S3Prefix',  
                                 split_type = 'Line',
                                 content_type = 'application/x-image',
                                 logs = False)